# 0. Prerequisites

In [1]:
!pip install gdown --quiet

In [72]:
from datasets import load_dataset
from nltk import sent_tokenize, word_tokenize
import random
import os
import sys

In [3]:
data = load_dataset('onestop_english')

Generating train split:   0%|          | 0/567 [00:00<?, ? examples/s]

Dataset onestop_english downloaded and prepared to /root/.cache/huggingface/datasets/onestop_english/default/1.1.0/6b19eec5680862ad1cf1990e98b06a98d1fa4c85f3585dc4dfab93f52b89d9cf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 567
    })
})

In [5]:
sentences_hard = []
sentences_simple = []
for i in range(len(data['train'])):
    if data['train'][i]['label'] == 2:
        for sentence in sent_tokenize(data['train'][i]['text']):
            sentences_hard.append(' '.join(word_tokenize(sentence.lower())))
    elif data['train'][i]['label'] == 0:
        for sentence in sent_tokenize(data['train'][i]['text']):
            sentences_simple.append(' '.join(word_tokenize(sentence.lower())))

In [6]:
!gdown https://drive.google.com/uc?id=13BC8BDL2MkksUtqsQ1clPxcPvKWgKlO3 --quiet
!gdown https://drive.google.com/uc?id=1zUMoJk3Jkf9wh40tRG1lSI07RDMp_l8n --quiet
!gdown https://drive.google.com/uc?id=19uhGuMkTeV_nveGQ2rbtbitQMI0b6Xsp --quiet
!gdown https://drive.google.com/uc?id=1bmw6wGVt5h52w3WEVVndrucEDBwws66l --quiet

In [7]:
def read_turkers(path):
    with open(path, 'r') as f:
        sentences = f.read().splitlines()
    return sentences

In [8]:
turkers_hard_one = read_turkers('/kaggle/working/test.8turkers.tok.norm.txt')
turkers_hard_two = read_turkers('/kaggle/working/tune.8turkers.tok.norm.txt')
turkers_simple_one = read_turkers('/kaggle/working/test.8turkers.tok.simp.txt')
turkers_simple_two = read_turkers('/kaggle/working/tune.8turkers.tok.simp.txt')

In [9]:
sentences_hard += turkers_hard_one
sentences_hard += turkers_hard_two
sentences_simple += turkers_simple_one
sentences_simple += turkers_simple_two

In [12]:
random.shuffle(sentences_hard)
random.shuffle(sentences_simple)

In [13]:
sentences_hard_train = sentences_hard[1001:len(sentences_hard)]
sentences_hard_test = sentences_hard[0:1000]

sentences_simple_train = sentences_simple[1001:len(sentences_simple)]
sentences_simple_test = sentences_simple[0:1000]

In [15]:
def write_to_file(sentences: list, filename: str):
    with open(filename, 'w') as f:
        for sentence in sentences:
            f.write(sentence.lower() + '\n')

In [16]:
write_to_file(sentences_hard_train, 'train_hard')
write_to_file(sentences_hard_test, 'test_hard')
write_to_file(sentences_simple_train, 'train_simple')
write_to_file(sentences_simple_test, 'test_simple')

In [17]:
VOCAB_DIRNAME = 'vocabularies' 

In [18]:
!git clone https://github.com/s-nlp/detox

Cloning into 'detox'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 180 (delta 50), reused 118 (delta 14), pack-reused 0
Receiving objects: 100% (180/180), 17.92 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [19]:
!pip install -r detox/requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
virtualenv 20.17.1 requires importlib-metadata>=4.8.3; python_version < "3.8", but you have importlib-metadata 3.10.1 which is incompatible.
pynndescent 0.5.8 requires importlib-metadata>=4.8.1; python_version < "3.8", but you have importlib-metadata 3.10.1 which is incompatible.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
lightning-utilities 0.7.1 requires importlib-metadata>=4.0.0; python_version < "3.8", but you have importlib-metadata 3.10.1 which is incompatible.
librosa 0.10.0 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
ibis-framework 2.1.1 requires importlib-metadata<5,>=4; python_version < "3.8", but you have importlib-m

In [33]:
def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [34]:
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/kaggle/working/detox/emnlp2021/style_transfer/condBERT


In [35]:
from condbert import CondBertRewriter
from choosers import EmbeddingSimilarityChooser
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

# 1. Loading BERT

In [36]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
import os
from tqdm.auto import tqdm, trange

In [37]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda:0')
device = torch.device('cpu')

In [38]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [39]:
model = BertForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
model.to(device);

# 2. Preparing the vocabularires.

In [41]:
hard_corpus_path = '/kaggle/working/train_hard'
simple_corpus_path = '/kaggle/working/train_simple'

In [42]:
if not os.path.exists(VOCAB_DIRNAME):
    os.makedirs(VOCAB_DIRNAME)

### 2.1 Preparing the DRG-like vocabularies

In [73]:
import argparse
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, hard_corpus, simple_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        hard_count_matrix = self.vectorizer.fit_transform(hard_corpus)
        self.hard_vocab = self.vectorizer.vocabulary_
        self.hard_counts = np.sum(hard_count_matrix, axis=0)

        simple_count_matrix = self.vectorizer.fit_transform(simple_corpus)
        self.simple_vocab = self.vectorizer.vocabulary_
        self.simple_counts = np.sum(simple_count_matrix, axis=0)

    def salience(self, feature, attribute='hard', lmbda=0.5):
        assert attribute in ['hard', 'simple']
        if feature not in self.hard_vocab:
            hard_count = 0.0
        else:
            hard_count = self.hard_counts[0, self.hard_vocab[feature]]

        if feature not in self.simple_vocab:
            simple_count = 0.0
        else:
            simple_count = self.simple_counts[0, self.simple_vocab[feature]]

        if attribute == 'hard':
            return (hard_count + lmbda) / (simple_count + lmbda)
        else:
            return (simple_count + lmbda) / (hard_count + lmbda)

In [44]:
from collections import Counter
c = Counter()

for fn in [hard_corpus_path, simple_corpus_path]:
    with open(fn, 'r') as corpus:
        for line in corpus.readlines():
            for tok in line.strip().split():
                c[tok] += 1

print(len(c))

22673


In [45]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

22673


In [46]:
with open(hard_corpus_path, 'r') as hard_corpus, open(simple_corpus_path, 'r') as simple_corpus:
    corpus_hard = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in hard_corpus.readlines()]
    corpus_simple = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in simple_corpus.readlines()]

In [47]:
hard_out_name = VOCAB_DIRNAME + '/hard-words.txt'
simple_out_name = VOCAB_DIRNAME + '/simple-words.txt'

In [48]:
threshold = 1

In [49]:
sc = NgramSalienceCalculator(corpus_hard, corpus_simple, False)
seen_grams = set()

with open(hard_out_name, 'w') as hard_out, open(simple_out_name, 'w') as simple_out:
    for gram in set(sc.hard_vocab.keys()).union(set(sc.simple_vocab.keys())):
        if gram not in seen_grams:
            seen_grams.add(gram)
            hard_salience = sc.salience(gram, attribute='hard')
            simple_salience = sc.salience(gram, attribute='simple')
            if hard_salience > threshold:
                hard_out.writelines(f'{gram}\n')
            elif simple_salience > threshold:
                simple_out.writelines(f'{gram}\n')

## 2.2 Evaluating word hardness with a logistic regression

In [50]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [51]:
X_train = corpus_hard + corpus_simple
y_train = [1] * len(corpus_hard) + [0] * len(corpus_simple)
pipe.fit(X_train, y_train);

In [52]:
coefs = pipe[1].coef_[0]
coefs.shape

(20502,)

In [53]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [54]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

## 2.3 Labelling BERT tokens by toxicity

In [55]:
from collections import defaultdict
hard_counter = defaultdict(lambda: 1)
simple_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_hard):
    for token in tokenizer.encode(text):
        hard_counter[token] += 1
for text in tqdm(corpus_simple):
    for token in tokenizer.encode(text):
        simple_counter[token] += 1

100%|██████████| 6938/6938 [00:04<00:00, 1430.44it/s]


In [56]:
token_hardness = [hard_counter[i] / (simple_counter[i] + hard_counter[i]) for i in range(len(tokenizer.vocab))]

In [57]:
with open(VOCAB_DIRNAME + '/token_hardness.txt', 'w') as f:
    for t in token_hardness:
        f.write(str(t))
        f.write('\n')

# 3. Setting up the model

### 3.1 Loading the vocabularies

In [58]:
with open(VOCAB_DIRNAME + "/hard-words.txt", "r") as f:
    s = f.readlines()
hard_words = list(map(lambda x: x[:-1], s))

with open(VOCAB_DIRNAME + "/simple-words.txt", "r") as f:
    s = f.readlines()
simple_words = list(map(lambda x: x[:-1], s))

In [59]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [60]:
token_hardness = []
with open(VOCAB_DIRNAME + '/token_hardness.txt', 'r') as f:
    for line in f.readlines():
        token_hardness.append(float(line))
token_hardness = np.array(token_hardness)
token_hardness = np.maximum(0, np.log(1/(1/token_hardness-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_hardness[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_hardness[tokenizer.encode(tok)][1] = 0

In [61]:
def adjust_logits(logits, label=0):
    return logits - token_hardness * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=hard_words,
    pos_words=simple_words,
    word2coef=word2coef,
    token_toxicities=token_hardness,
    predictor=predictor,
)

In [62]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

2023-03-26 10:36:17,359 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmplaajkwog


100%|██████████| 160000128/160000128 [00:08<00:00, 19994735.82B/s]

2023-03-26 10:36:25,766 copying /tmp/tmplaajkwog to cache at /root/.flair/embeddings/glove.gensim.vectors.npy
2023-03-26 10:36:25,944 removing temp file /tmp/tmplaajkwog


2023-03-26 10:36:26,495 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmph0wofe3t


100%|██████████| 21494764/21494764 [00:01<00:00, 12241780.88B/s]

2023-03-26 10:36:28,673 copying /tmp/tmph0wofe3t to cache at /root/.flair/embeddings/glove.gensim
2023-03-26 10:36:28,707 removing temp file /tmp/tmph0wofe3t


In [63]:
print(editor.translate('so social media, which can open you up to the scrutiny and analysis of others, is not that appealing to her.', prnt=False))

so webmedia , which can open you about to the observation and observation of others , is not that attractive to her .


In [64]:
print(editor.replacement_loop('so social media, which can open you up to the scrutiny and analysis of others, is not that appealing to her.', verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))

but studying different information information , because they do not opens many things because they respond more quickly too quickly because they make judgment mistakes because they analyze too many different things , does not make someone more interesting because they study many things .


In [66]:
!tar -czf data.tar.gz /kaggle/working/vocabularies

tar: Removing leading `/' from member names


In [67]:
os.chdir(r'/kaggle/working')

In [68]:
from IPython.display import FileLink
FileLink(r'data.tar.gz')

/kaggle/working/data.tar.gz

In [116]:
predictor = MaskedTokenPredictorBert(
    model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, 
    confuse_bert_args=False)
editor.predictor = predictor

def adjust_logits(logits, label=0):
    return logits - editor.token_toxicities * 10

predictor.logits_postprocessor = adjust_logits

cho = EmbeddingSimilarityChooser(sim_coef=100, tokenizer=tokenizer)

In [129]:
model_outputs = []
for i, sentence in enumerate(tqdm.tqdm(sentences_hard_test)):
    out = editor.translate(sentence, prnt=False)
    model_outputs.append(out)

100%|██████████| 1000/1000 [02:57<00:00,  5.65it/s]


In [130]:
with open('model_outputs', 'w') as o:
    for output in model_outputs:
        o.write(output)